In [1]:
import numpy as np
import os
import pandas as pd

os.environ["HF_ENDPOINT"] = "https://huggingface.co"
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/home/andi/.pyenv/versions/3.8.12/envs/simply-japanese/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-23 21:47:54.685659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 21:47:54.786842: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 21:47:54.786863: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
tokenizer = AutoTokenizer.from_pretrained("astremo/friendly_JA")
model = AutoModelForSeq2SeqLM.from_pretrained("astremo/friendly_JA", torchscript = True)

In [5]:
text = "その会社の税引き前利益は２億５０００万ドルだった。"
input_ids = tokenizer(text, return_tensors="pt").input_ids
attention_mask = input_ids.ne(model.config.pad_token_id).long()
decoder_input_ids = tokenizer(f'<pad>{text}', return_tensors='pt').input_ids
traced_model = torch.jit.trace(model, (input_ids, attention_mask, decoder_input_ids))
model.eval()
with torch.no_grad():
    outputs = model.generate(input_ids)

In [6]:
tokenizer.decode(outputs.squeeze(), skip_special_tokens=True)

'その会社の税のコストは2億5000万ドルだった。'